
#  高性能プログラミングと性能測定(5) --- 練習問題 (SIMD + ILP + マルチコア)


# 1. 1/8球の体積 を求めるプログラム (SIMD + ILP + マルチコア)


# <font color="green"> Problem 1 :  SIMD, ILP, マルチコア並列化</font>
* cs03で行った1/8球の体積を求めるプログラムを, SIMD, ILP, マルチコアすべてを利用して高速化せよ


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile omp_ball_simd_ilp_mp.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>
#include <unistd.h>

double volume_of_ball(long n, int nteams, int nthreads) {
  double h = 1.0 / (double)n;
  long s = 0;
  for (long i = 0; i < n; i++) {
    for (long j = 0; j < n; j++) {
      for (long k = 0; k < n; k++) {
        double x = (i + 0.5) * h;
        double y = (j + 0.5) * h;
        double z = (k + 0.5) * h;
        s += (x * x + y * y + z * z < 1.0);
      }
    }
  }
  return s * h * h * h;
}

int main(int argc, char ** argv) {
  long n           = (1 < argc ? atol(argv[1]) : 100);
  char * nteams_   = getenv("OMP_NUM_TEAMS");
  int    nteams    = (nteams_   ? atoi(nteams_) : 1);
  char * nthreads_ = getenv("OMP_NUM_THREADS");
  int    nthreads  = (nthreads_ ? atoi(nthreads_) : 1);

  printf("n             : %ld\n", n);
  printf("nteams        : %d\n", nteams);
  printf("nthreads      : %d\n", nthreads);
  /* 計測開始 */
  double t0 = omp_get_wtime();
  /* 計算本体 */
  double v = volume_of_ball(n, nteams, nthreads);
  /* 計測終了 */
  double t1 = omp_get_wtime();
  double dt = t1 - t0;          /* sec */
  double error = fabs(v - M_PI/6.0);
  if (error > 1.0e-2) {
    fprintf(stderr, "WARNING: error (%f) > 0.01\n", error);
    fprintf(stderr, "check your program\n");
  }
  printf("volume        : %.9f\n", v);
  printf("error         : %e\n", error);
  printf("elapsed       : %7.3f\n", dt);
  printf("n^3 / nsec    : %7.3f\n",
         (double)n * (double)n * (double)n / dt * 1.0e-9);
  return 0;
}


* ヒント
  * このプログラムのベクトル化で一つ厄介なのは, `x * x + y * y + z * z < 1.0` の部分
  * `A < B` は, 満たされていれば1, なければ0という式
  * ここで`A, B` がベクトル型だったら `A < B` が 「`A[i] < B[i]`を満たす要素の数」だったりしたら話が早いのだがそうは問屋がおろさず, そもそもそのような演算子 (ベクトル型同士の大小比較) というものが許されていない
  * したがって, `count_lt(double A, double B)` とでも名付けて, 上記の値を返す関数を自分で作る必要がある

In [ ]:
nvc -fast -mp=multicore omp_ball_simd_ilp_mp.c -o omp_ball_simd_ilp_mp.exe

* 以下の`OMP_NUM_THREADS=1` を色々変えて実行してみよ
* なお,
  * https://tauleg.zapto.org:8000/ には18の(仮想)コア
  * https://taulec.zapto.org:8000/ には152の(仮想)コア
があるので, できれば後者で, 多数のスレッド数でやってみよ  

In [ ]:
OMP_NUM_THREADS=1 ./omp_ball_mp.exe 512

* 以下を適切に修正して様々なスレッド数で実行し, 

In [ ]:
BEGIN SOLUTION
END SOLUTION
for th in 1 2 適切にスレッド数を並べよ ; do
  echo "====="
  OMP_PROC_BIND=true OMP_NUM_THREADS=${th} ./omp_ball_mp.exe 512
done

* 結果を以下にコピペして性能向上を可視化せよ(性能向上がすぐに頭打ちになるようであれば, $n$の値を調節せよ)
* GPUでの結果と比較せよ

In [ ]:
import re
import matplotlib.pyplot as plt

DATA = r"""
=====
n             : 512
nteams        : 1
nthreads      : 1
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.885
=====
n             : 512
nteams        : 1
nthreads      : 2
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.876
=====
n             : 512
nteams        : 1
nthreads      : 4
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.882
=====
n             : 512
nteams        : 1
nthreads      : 6
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.871
=====
n             : 512
nteams        : 1
nthreads      : 8
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.882
=====
n             : 512
nteams        : 1
nthreads      : 9
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.877
=====
n             : 512
nteams        : 1
nthreads      : 12
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.883
=====
n             : 512
nteams        : 1
nthreads      : 18
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.046
n^3 / nsec    :   2.888
"""

def put_rec(R, D):
    num_teams = int(D["nteams"])
    num_threads = int(D["nthreads"])
    perf = float(D["n^3 / nsec"])
    R.append((num_teams * num_threads, perf))

def speedup(data):
    data = data.strip().split("\n")
    R = []                      # (nteams * nthreads, n^3/nsec)
    D = None
    pat = re.compile(r"(?P<k>[^:]+?) *: +(?P<v>\d+(\.\d+)?)")
    for line in data:
        if line.strip() == "=====":
            if D is not None:
                put_rec(R, D)
            D = {}
        else:
            m = pat.match(line)
            key = m.group("k")
            val = m.group("v")
            D[key] = val
    if D is not None:
        put_rec(R, D)
    plt.ylabel("n^3/nsec")
    plt.xlabel("num_teams * num_threads")
    R.sort()
    X = [x    for x, perf in R]
    Y = [perf for x, perf in R]
    L = [Y[0]/X[0] * x for x in X]
    plt.plot(X, Y)
    plt.plot(X, L)
    plt.show()

speedup(DATA)